In [1]:
# !pip install pydot

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

#### LOAD DATASET

In [3]:
# helper functions
def DataLoading (mypath):
    print ("Loading the data")
    dataframe = pd.read_csv(mypath,header = None,engine = 'python',sep=",")
    return dataframe


# column names
def prepate_dataset(df):
  column_names = ["duration", "protocol_type", "service", "flag" ,"src_bytes",
                  "dst_bytes", "land", "wrong_fragment", "urgent", "hot", "num_failed_logins",
                  "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root", 
                  "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds",
                  "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", 
                  "srv_error_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", 
                  "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
                  "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
                  "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate",
                  "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "attack", "difficulty"]

  # assigning the column names
  df.columns = column_names

  # preparing class labels 
  df['class'] = df['attack'].replace(['back', 'land', 'neptune', 'pod','smurf', 'teardrop'],'attack') # Dos
  df['class'] = df['class'].replace(['ipsweep', 'nmap','portsweep', 'satan'],'attack') # Probe
  df['class'] = df['class'].replace(['buffer_overflow','loadmodule', 'perl','rootkit'],'attack') # U2R
  df['class'] = df['class'].replace(['ftp_write', 'guess_passwd','imap', 'multihop', 'phf','spy', 'warezclient','warezmaster'],'attack') # R2L

  # droping attacks column
  df = df.drop(['attack'], axis=1)
  df = df.drop(['difficulty'], axis=1)
  # Creating a instance of label Encoder.
  le = LabelEncoder()
  
  # Using .fit_transform function to fit label
  # encoder and return encoded label
  df['protocol_type'] = le.fit_transform(df['protocol_type'])
  df['service'] = le.fit_transform(df['service'])
  df['flag'] = le.fit_transform(df['flag'])
  df['class'] = le.fit_transform(df['class'])

  return df



In [4]:
df_train = DataLoading("./NSL-KDD/KDDTrain+.txt")
df_train = prepate_dataset(df_train)


df_val = DataLoading("./NSL-KDD/KDDTrain+_20Percent.txt")
df_val = prepate_dataset(df_val)

Loading the data
Loading the data


#### Pre-Precess DATA

In [5]:
# divide to train and test arrays 

x_train = df_train.drop(['class'], axis=1).to_numpy()
y_train = df_train['class'].to_numpy()

x_test = df_val.drop(['class'], axis=1).to_numpy()
y_test = df_val['class'].to_numpy()

print(x_train.shape, y_train.shape,  x_test.shape, y_test.shape)

(125973, 41) (125973,) (25192, 41) (25192,)


In [6]:
# normalize the data

norm = np.linalg.norm(x_train)
x_train = x_train/norm

norm = np.linalg.norm(x_test)
x_test = x_test/norm

In [7]:
# reshape to fit in gru

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

#### BiGRU

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU, Bidirectional, Flatten,Embedding
from keras.utils import plot_model

2022-09-30 05:25:00.190754: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model = Sequential()
model.add(Bidirectional(GRU(64, return_sequences=True),input_shape=(41, 1)))
model.add(Dropout(0.2))


model.add(Bidirectional(GRU(32)))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

2022-09-30 05:25:03.404138: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 41, 128)          25728     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 41, 128)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               31104     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

In [13]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(x_train,
          y_train,
          epochs=100,
          validation_data=(x_test, y_test))


Epoch 1/100
3937/3937 [==============================] - 130s 32ms/step - loss: 0.6724 - accuracy: 0.1016 - val_loss: 0.6757 - val_accuracy: 0.1031
Epoch 2/100
3937/3937 [==============================] - 133s 32ms/step - loss: 0.6654 - accuracy: 0.1055 - val_loss: 0.6738 - val_accuracy: 0.1112
Epoch 3/100
3937/3937 [==============================] - 135s 32ms/step - loss: 0.6589 - accuracy: 0.1124 - val_loss: 0.6676 - val_accuracy: 0.1113
Epoch 4/100
3937/3937 [==============================] - 139s 32ms/step - loss: 0.653 - accuracy: 0.1192 - val_loss: 0.6669 - val_accuracy: 0.1143
Epoch 5/100
3937/3937 [==============================] - 133s 32ms/step - loss: 0.6511 - accuracy: 0.129 - val_loss: 0.6667 - val_accuracy: 0.1229
Epoch 6/100
3937/3937 [==============================] - 133s 32ms/step - loss: 0.6412 - accuracy: 0.1385 - val_loss: 0.6656 - val_accuracy: 0.1238
Epoch 7/100
3937/3937 [==============================] - 132s 32ms/step - loss: 0.636 - accuracy: 0.139 - val_loss